In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from math import sqrt
from random import randrange


np.random.seed(6201)
mean1 = [-1, -1]
mean2 = [1, 1]
cov = [[1, 0.25], [0.25, 1]]
X1 = np.random.multivariate_normal(mean1, cov, 200)
y1 = np.ones(len(X1))
X2 = np.random.multivariate_normal(mean2, cov, 200)
y2 = np.ones(len(X2)) * -1


def split(dataset, folds):

      dataset_split = list()
      dataset_copy = list(dataset)
      fold_size = int(len(dataset) / folds)
      for i in range(folds):
        fold = list()
        while len(fold) < fold_size:
          index = randrange(len(dataset_copy))
          fold.append(dataset_copy.pop(index))
          dataset_split.append(fold)

      return dataset_split


def leaveOneOut(X1, y1, X2, y2):

      X1_train = np.asarray(split(X1, 200)).reshape(200, -1)
      y1_train = np.asarray(split(y1, 200)).reshape(200, )
      X2_train = np.asarray(split(X2, 200)).reshape(200, -1)
      y2_train = np.asarray(split(y2, 200)).reshape(200, )

      X_train = np.vstack((X1_train, X2_train))
      y_train = np.hstack((y1_train, y2_train))

      X1_test = np.asarray(split(X1, 200)).reshape(200, -1)
      y1_test = np.asarray(split(y1, 200)).reshape(200, )
      X2_test = np.asarray(split(X2, 200)).reshape(200, -1)
      y2_test = np.asarray(split(y2, 200)).reshape(200, )

      X_test = np.vstack((X1_test, X2_test))
      y_test = np.hstack((y1_test, y2_test))
      
      return X_train, y_train, X_test, y_test

#X_train, y_train, X_test, y_test = leaveOneOut(X1,y1,X2,y2)

from keras.datasets import mnist
from keras.utils import np_utils


(X_train, y_train), (X_test, y_test) = mnist.load_data()

num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape((X_train.shape[0], num_pixels)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], num_pixels)).astype('float32')

X_train, X_test = X_train / 255, X_test / 255
X_test = X_test / 255

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

y_train_0 = (y_train == 0)
y_train_1 = (y_train == 1)
y_test_0 = (y_test == 0)
y_test_1 = (y_test == 1)


from sklearn.svm import SVC
clf = SVC(kernel = 'linear', C = 10 )
clf.fit(X_train, y_train_0)
svcount= len(clf.support_)
print('w = ', clf.coef_)
print('b = ', clf.intercept_)
print('Indices of support vectors = ', clf.support_)
print("Number of support vectors = ", svcount)
print('Support vectors = ', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)
print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))

y_predict = clf.predict(X_test)

#leave-one-out
y_true, y_pred = list(), list()
y_true.append(y_test[0])
y_pred.append(y_predict[0])
acc = accuracy_score(y_true, y_pred)
print("Accuracy: " , acc)

cv = LeaveOneOut()
       
scores = cross_val_score(clf, X1, X2, scoring = 'neg_mean_absolute_error', cv = cv, n_jobs = -1)
print(np.mean(np.absolute(scores)))



cm = confusion_matrix(y_test, y_predict)
accuracy_score(y_test,y_predict)

from matplotlib.colors import ListedColormap
X_set, y_set = X_test, y_test
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, clf.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('SVM (Test set)')
plt.legend()
plt.show()

ValueError: ignored

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns